In [2]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 

import scipy.stats as spt
import numpy as np

import matplotlib.pyplot as plt

In [3]:
# 단일 모집단 검정
import rpy2.robjects as robjs

rng = np.random.default_rng()
rvs = spt.norm.rvs(loc=5, scale=10, size=(50, 1), random_state=rng)
ss=spt.ttest_1samp(rvs, 5.0)
print(ss)
print(spt.ttest_1samp(rvs, 0.0))

robjs.r['load']("class3.RData")
ex1=robjs.r['exa10_9']

tt=spt.ttest_1samp(ex1,np.mean(ex1))

ModuleNotFoundError: No module named 'rpy2'

In [4]:
# 카이제곱 검정 (적합도)

print(spt.chisquare([16, 18, 16, 14, 12, 12]))

print(spt.chisquare([16, 18, 16, 14, 12, 12], f_exp=[16, 16, 16, 16, 16, 8]))

print(spt.chisquare([16, 18, 16, 14, 12, 12], ddof=[0,1,2]))

obs=np.array([31,26,22,18,13,10])
print(spt.chisquare(obs))

# 교차분석

ss=robjs.r['exa12_5']
print(ss)

ch2,pval,df,evl=spt.chi2_contingency(observed=ss)



Power_divergenceResult(statistic=2.0, pvalue=0.8491450360846096)
Power_divergenceResult(statistic=3.5, pvalue=0.6233876277495822)
Power_divergenceResult(statistic=2.0, pvalue=array([0.84914504, 0.73575888, 0.5724067 ]))
Power_divergenceResult(statistic=15.7, pvalue=0.007754884452437287)


NameError: name 'robjs' is not defined

In [ ]:
# One way ANOVA

np.random.seed(5)

grp1=np.random.normal(3,4,30)
grp2=np.random.normal(3.1,4.1,30)
grp3=np.random.normal(2.84,4.3,30)

grp=[grp1,grp2,grp3]

import seaborn as sb

sb.kdeplot(grp1)
sb.kdeplot(grp2)
sb.kdeplot(grp3)
plt.show()

sb.boxplot(data=grp)
plt.show()

print(spt.f_oneway(grp1,grp2,grp3))
F_stat,pval=spt.f_oneway(*grp)

ttt=robjs.r['exa13_6']

print(spt.f_oneway(ttt[0],ttt[1],ttt[2]))

In [1]:
import scipy.stats as spt
import numpy as np
import pandas as pd
import seaborn as sb
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.weightstats as sw
import matplotlib.pyplot as plt


In [2]:
# Two sample t test

fres = np.array([3.7,4.3,2.5,3.3,3.6,3.1])
soph = np.array([1.8,4.2,4.1,2.2,3.2,3.8])
twosam = spt.ttest_ind(fres,soph)
print(twosam)

print('t statistic = %.3f, p-value = %.3f'%(twosam))


print('1학년 :', np.mean(fres))
print('2학년 :', np.mean(soph))

Ttest_indResult(statistic=0.41473113113479565, pvalue=0.6870956353065248)
t statistic = 0.415, p-value = 0.687
1학년 : 3.4166666666666674
2학년 : 3.216666666666667


In [3]:
# Dataframe

data1=pd.DataFrame(data=np.array([[3.7,1.8,3.3,4.1],[4.3,4.2,3.7,3.8],[2.5,4.1,3.4,3.5],[3.3,2.2,3.9,3.2],[3.6,3.2,np.nan,2.3]]),
                   columns=['fre','sop','jun','sen'])
print(data1['jun'],'\n')

sop = data1['sop']
jun = data1['jun']
jun = jun.fillna(jun.mean())

print(jun,'\n')

tsam = spt.ttest_ind(sop,jun)
print(tsam)

0    3.3
1    3.7
2    3.4
3    3.9
4    NaN
Name: jun, dtype: float64 

0    3.300
1    3.700
2    3.400
3    3.900
4    3.575
Name: jun, dtype: float64 

Ttest_indResult(statistic=-0.9550271234200176, pvalue=0.36752757550294646)


In [4]:
# Paired t test

before=np.array([68,61,60,68,67,64,66,67,66,67,72,74,61,71,58,77])
after=np.array([56,55,67,62,59,67,50,60,59,53,60,65,62,61,64,57])

pairsam = spt.ttest_rel(before,after)
print(pairsam)

Ttest_relResult(statistic=3.562626515972126, pvalue=0.002834843356363954)


In [5]:
# Statsmodels를 이용한 t test

fre=data1['fre']
sen=data1['sen']


# 가설에는 two-sided / larger / smaller 3가지, 분산에는 pooled, unequal 2가지, value는 두 집단의 평균 차이가 얼마인지 나타내는 수
tsams=sw.ttest_ind(fre,sen,alternative='two-sided',usevar='pooled',value=0)
print('tstat = \t',tsams[0],'\np-val = \t',tsams[1],'\ndegree of freedom = \t',tsams[2])



# 쌍체검정 (One-sided)

ptsams = sw.ttost_paired(before,after,0.1,0.2)
print(ptsams)

tstat = 	 0.23453251491009644 
p-val = 	 0.8204627528467475 
degree of freedom = 	 8.0
(0.9982467120464241, (3.510806493921623, 0.0015764190017186505, 15.0), (3.458986471871119, 0.9982467120464241, 15.0))


In [6]:
# Two-way ANOVA from statsmodels // Referred : https://www.statsmodels.org/stable/anova.html#module-statsmodels.stats.anova

moore = sm.datasets.get_rdataset("Moore", "carData", cache=True)

data2 = moore.data

data2 = data2.rename(columns={"partner.status":"partner_status"})

moore_lm = ols('conformity ~ C(fcategory, Sum)*C(partner_status, Sum)',data=data2).fit()

table = sm.stats.anova_lm(moore_lm, typ=2)

print(table)

                                              sum_sq    df          F  \
C(fcategory, Sum)                          11.614700   2.0   0.276958   
C(partner_status, Sum)                    212.213778   1.0  10.120692   
C(fcategory, Sum):C(partner_status, Sum)  175.488928   2.0   4.184623   
Residual                                  817.763961  39.0        NaN   

                                            PR(>F)  
C(fcategory, Sum)                         0.759564  
C(partner_status, Sum)                    0.002874  
C(fcategory, Sum):C(partner_status, Sum)  0.022572  
Residual                                       NaN  
